In [1]:
from utils import *

In [ ]:
# Ask user for league to scrape

# Main Script
if __name__ == '__main__':
    # Get team name from user
    league = input("Enter the league: ")
    gender = input("Enter the competition gender (M/F): ")
    folder_path = os.path.join(os.getcwd(), "Competitions")
    os.makedirs(folder_path, exist_ok=True)  # Create the folder if it doesn't exist
    # File-based caching
    LEAGUES_CACHE = folder_path + '\league_links.json'
    closest_league, league_info = get_closest_league(league,LEAGUES_CACHE,gender)

    if closest_league:
        print(f"Closest match: {closest_league}, URL: {league_info['url']}")
    else:
        print("No close match found.")

    # Select season 
    season = input("Select the season")
    # get selected competition folder (or create it)
    if gender=='M':
        full_gender = 'Men'
    else:
        full_gender = 'Women'
    folder_path = os.path.join(os.getcwd(), closest_league + '-' + full_gender)
    os.makedirs(folder_path, exist_ok=True)  
    SEASONS_CACHE = os.path.join(folder_path, f'{closest_league}_seasons.json')
    season_url = get_season_url(season,SEASONS_CACHE,league_info['url'])

    if season_url:
        print(f"Closest match: {season}, URL: {season_url[1]}")
    else:
        print("Season URL not found")
    
    # Get tables from the competition page
    competition_url = season_url[1]
    folder_path = folder_path + f"\{season}"
    os.makedirs(folder_path, exist_ok=True)
    output_file_for = folder_path + "\Season-Stats.xlsx"
    table_card_position = "left"
    scrape_page_tables(competition_url, output_file_for,table_card_position)
    print(f"{closest_league} data for team on the {season} saved in {output_file_for}.")

    respect_fbref_scrape_policy() # 6 seconds timeout between requests

    output_file_vs = folder_path + "\Season-Stats-against.xlsx"
    table_card_position = "right"
    scrape_page_tables(competition_url, output_file_vs,table_card_position)
    print(f"{closest_league} data vs team on the {season} saved in {output_file_vs}.")

    # Move to the Fixture page
    table_card_position = "left"
    output_file = folder_path + "\Fixtures.xlsx"
    fixtures_url = get_scores_and_fixtures_url(competition_url)
    print(f"Fixture URL: {fixtures_url}")
    scrape_page_tables(fixtures_url,output_file,table_card_position)

    # Eliminate empty rows
    df = pd.read_excel(output_file, engine='openpyxl')
    df_cleaned = df.dropna(how='all')
    df_cleaned.to_excel(output_file, index=False, engine='openpyxl')
    print(f"Cleaned Excel file saved to {output_file}")
